# Structural Estimation of Agricultural Household Model (Monte Carlo Simulation)

In this document, I will conduct a Monte Carlo Simulation of Agricultural Household Model.

## Model

I consider a model in which a farmer maximizes lifetime expected utility by making decisions in agricultural production and household (ie. consumption).
More specifically, the farmer solves the following maximization problem by deciding (i) consumption, (ii) production input, and (iii) saving:

\begin{align*}
\max_{\{C_t > 0, M_t > 0, S_t \ge 0\}_{t=0}^{\infty}} \quad U(W_t) &= \sum_{t=0}^{\infty} E_0 \left[\frac{C_t^{1 - \rho}}{1 - \rho} \right] \\
s.t. \quad C_t + M_t + S_t &= W_t \quad \text{(budget constraint)} \\
     \quad Y_{t+1} &= M_t^{\beta_m} \varepsilon_{t+1} \quad \text{(production function)} \\
     \quad W_{t+1} &= Y_{t+1} + Z_{t+1} + (1 + r) S_t \quad \text{(wealth transition)}
\end{align*}


where
- $C_t$: Consumption, $S_t$: saving, $M_t$: Production inputs, $W_t$: Wealth
- $Y_t$: Business revenue, $Z_t \sim \log N(\mu_Z, \sigma_Z^2)$: Other income
- $\varepsilon_t \sim \log N(\mu_{\varepsilon}, \sigma_{\varepsilon}^2)$: Productivity shocks.

The Bellman equation is

\begin{align*}
V(W) &= \max_{C > 0, M > 0, S \ge 0} \frac{C^{1 - \rho}}{1 - \rho} + \beta E[V(W')] \\
s.t. \quad C + M + S &= W \quad \text{(budget constraint)} \\
     \quad Y' &= M^{\beta_m} \varepsilon' \quad \text{(production function)} \\
     \quad W' &= Y' + Z' + (1 + r) S \quad \text{(wealth transition)}
\end{align*}

The first order conditions are

\begin{align*}
C^{-\rho} &= \beta \beta^m M^{\beta_m - 1} E[\varepsilon' V' (M^{\beta_m} \varepsilon' + Z' + (1 + r) S)], \\
C^{-\rho} &= \beta (1 + r) E[V' (M^{\beta_m} \varepsilon' + Z' + (1 + r) S)] + \phi, \\
\phi S &= 0,
\end{align*}

where $\phi$ is the Lagrangean multiplier on $S \ge 0$, and the envelope condition is
\begin{equation*}
V'(W) = (W - S - M)^{-\rho}.
\end{equation*}

Note that there is an uncertainty in production but there is no insurance for that risk.
Hence, separability does not hold and the household preference (in this case, risk preference, $\rho$) affects production decision making (in this case, production input, $M$).
Also, notice that it is assumed that there is a liquidity constraint and the household cannot borrow money.
A reason indeed is that without a liquidity constraint, I found that there is almost no variation in business inputs across wealth levels.
My guess is that, while the first order conditions tell that $M$ can be different from the profit-maximizing level due to correlation between $\varepsilon'$ and $V'(W')$, $V'(W')$ is almost linear in my parameter setting. 

## Observables

I assume that the following variables are included in the dataset at hand:

- Consumption, $C_t$;
- Business input, $M_t$;
- Business revenue, $Y_t$;
- Wealth, $W_t$;
- Interest income $(1 + r) S$.

Notice that saving $S$ is not directly observed.

## Estimation procedure

Parameters to be estimated are $\theta = (\rho, \beta, \beta_m, \mu_{\varepsilon}, \sigma_{\varepsilon}, \mu_Z, \sigma_Z, r)$.
Let $\theta_1 = (\rho, \beta)$ and $\theta_2 = (\beta_m, \mu_{\varepsilon}, \sigma_{\varepsilon}, \mu_Z, \sigma_Z, r)$

I first estimate a subset of parameters, $\theta_2$, with reduced form analyses:

- $(\beta_m, \mu_{\varepsilon}, \sigma_{\varepsilon})$: I regress $\log(Y_{t+1})$ on $\log(M_t)$ with a constant, and the coefficient of $\log(M_t)$ is $\widehat{\beta_m}$.
The estimated intercept is $\widehat{\mu_{\varepsilon}}$ and the variance of residual ($Y_{t+1} - \widehat{\mu_{\varepsilon}} - \widehat{\beta_m} \log(M_t)$) is $\widehat{\sigma_{\varepsilon}}^2$.
- $(r)$: I obtain $\widehat{r}$ as an average of $\frac{\text{interest income}}{S_t}$.
- $(\mu_Z, \sigma_Z)$: I obtain $Z_t$ as $W_t - Y_t - (\text{interest income})_t$ and the mean and variance of $\log(Z_t)$ are $\widehat{\mu_Z}$ and $\widehat{\sigma_Z}^2$, respectively.

I use the indirect inference to estimate the rest of the parameters, $\theta_1$.
Estimation involves the following steps:

1. Given a set of parameters ($\theta_1, \widehat{\theta_2}$), solve the dynamic programming model and obtain policy functions.
2. Simulate the data and obtain simulated moments ($m(\theta_1, \widehat{\theta_2})$).
3. Obtain the distance between simulated moments, $m(\theta_1, \widehat{\theta_2})$, and data moments ($m$): $D(\theta_1, \widehat{\theta_2}) = (m(\theta_1, \widehat{\theta_2}) - m)' (m(\theta_1, \widehat{\theta_2}) - m)$.
4. Estimate $\theta_1$ as the solution of $D(\theta_1, \widehat{\theta_2}) = 0$.

As moment conditions, I use the followings:

- $E[E[C | W] - C] = 0$,
- $E[E[S | W] - S] = 0$.

The degree of risk aversion, $\rho$, is identified through how the household considers the trade-off between safe investment ($S$) and risky investment ($M$), given their respective returns.
The discount factor, $\beta$, is identified through how the household considers the trade-off between today's consumption ($C$) and future returns ($M$ and $S$).
Since there are two parameters and two moments, the parameters are just-identified.

For inference, the standard errors of $\theta_2$ are obtained with an ordinary method (eg. standard errors in regressions).
The standard errors of $\theta_1$, on the other hand, need to account for estimated parameters, $\widehat{\theta_2}$, in the estimation process.
Hence, I use bootstrap method to obtain the standard errors of $\theta_1$.
One alternative is to estimate all parameters at once by using all moment conditions.
While this yields more efficient estimates and the complication of two-step estimators can be ignored, this is more computationally intensive due to more parameters to estimate in one step.

### More detailed procedure in step 1

#### 1. Solve Bellman equation by policy function iteration

Notice that the Bellman equation can be expressed as an equation with one state variable and two control variables:

\begin{equation*}
V(W) = \max_{C > 0, S \ge 0} \frac{C^{1 - \rho}}{1 - \rho} + \beta E[V((W - C - S)^{\beta_m} \varepsilon' + Z' + (1 + r) S)].
\end{equation*}

The expectation is taken over two random shocks: productivity shock $\varepsilon'$, and random income shock $Z'$.

I approximate the value function $V(\cdot)$ by the collocation method (Miranda and Fackler (2002), for example).
That is, I approximate $V(W) \approx \sum_{j = 1}^n c_j \phi_j(W)$.
Using this approximation, the Bellman equation above is expressed as

\begin{equation*}
\sum_{j = 1}^n c_j \phi_j(W) = \max_{C > 0, S \ge 0} \frac{C^{1 - \rho}}{1 - \rho} + \beta E \left[ \sum_{j = 1}^n c_j \phi_j((W - C - S)^{\beta_m} \varepsilon' + Z' + (1 + r) S) \right].
\end{equation*}

I also approximate the expectation using the Gaussian quadrature method:

\begin{equation*}
\sum_{j = 1}^n c_j \phi_j(W) = \max_{C > 0, S \ge 0} \frac{C^{1 - \rho}}{1 - \rho} + \beta \sum_{k = 1}^K w_k \left[ \sum_{j = 1}^n c_j \phi_j((W - C - S)^{\beta_m} \varepsilon_k' + Z_k' + (1 + r) S) \right],
\end{equation*}
where $w_k$ is a Gaussian quadrature weight.

Given this approximation, I solve the Bellman equation with the policy function iteration method:

1. Set the initial guess of consumption and saving at each collocation node, $(C_s, S_s)$, where $s$ is a collocation node.
2. At each collocation node, obtain 
\begin{equation*}
v_s = \frac{C_s^{1 - \rho}}{1 - \rho} + \beta \sum_{k = 1}^K w_k \left[ \sum_{j = 1}^n c_j \phi_j((W_s - C_s - S_s)^{\beta_m} \varepsilon_k' + Z_k' + (1 + r) S_s) \right].
\end{equation*}
3. Update the collocation coefficients $c_j$ by solving $\sum_{j = 1}^n c_j \phi_j (W_s) = v_s$ for all collocation nodes $s$.
4. With an updated $c_j$, calculate
\begin{equation*}
\frac{C_s^{1 - \rho}}{1 - \rho} + \beta \sum_{k = 1}^K w_k \left[ \sum_{j = 1}^n c_j \phi_j((W_s - C_s - S_s)^{\beta_m} \varepsilon_k' + Z_k' + (1 + r) S_s) \right].
\end{equation*}
and find $C_s$ and $S_s$ that maximize this.
5. Iterate steps 2-4 until $(C_s, S_s)$ are converged.

Given obtained $(C_s, S_s)$, we approximate the policy functions:
\begin{align*}
C(W) &\approx \sum_{j = 1}^n c_j^c \phi(W) \\
S(W) &\approx \sum_{j = 1}^n c_j^s \phi(W).
\end{align*}
These are used to construct moment conditions.

## Codes

Now the codes to estimate parameters and conduct a Monte Carlo(-like) simulation are shown.
First, I import packages:

In [33]:
# using Pkg;
# Pkg.add("PyPlot");
# Pkg.add("Distributions");
# Pkg.add("QuantEcon");
# Pkg.add("LaTeXStrings");
# Pkg.add("Optim");
# Pkg.add("ReadStat");
# Pkg.add("LinearAlgebra");
# Pkg.add("Interpolations");
# Pkg.add("FileIO");
# Pkg.add("StatFiles");
# Pkg.add("DataFrames");
# Pkg.add("CSV");
# Pkg.add("NamedArrays");
# Pkg.add("SharedArrrays");
# Pkg.add("DelimitedFiles");
# Pkg.add("Random");
# Pkg.add("CompEcon");
# Pkg.add("MultivariateStats");

using PyPlot
using Distributions
using QuantEcon
using LaTeXStrings
using Optim
using Optim: optimize
using ReadStat
using Interpolations
using FileIO, StatFiles, DataFrames
using CSV
using LinearAlgebra
using NamedArrays
using SharedArrays
using DelimitedFiles
using Random
using CompEcon
using MultivariateStats
using JuMP
using Ipopt

### Functions to obtain policy functions

Here, I define functions to obtain policy functions.
These functions are used both for generating data and for estimating parameters.

First, I define a function to calculate 
\begin{equation*}
v_s = \frac{C_s^{1 - \rho}}{1 - \rho} + \beta \sum_{k = 1}^K w_k \left[ \sum_{j = 1}^n c_j \phi_j((W_s - C_s - S_s)^{\beta_m} \varepsilon_k' + Z_k' + (1 + r) S_s) \right]
\end{equation*}
at each collocation node, given $(C_s, S_s)$:


In [34]:
# Define a function to calculate the value function given (consumption, saving).
function calculate_vs(C, S, W, c, basis, epsilon, Z, w, rho, beta, beta_m, r)
    return C ^ (1 - rho) / (1 - rho) + beta * dot(funeval(c, basis, [(W - C - S) ^ beta_m * epsilon + Z .+ (1 + r) * S])[1], w)
end

calculate_vs (generic function with 1 method)

I then define a function to update the collocation coefficients, $c$:

In [35]:
# Define a function to calculate the new collocation coefficients given v_s.
function update_c(W, vs, basis)
    return c = funfitxy(basis, W, vs)[1]
end

update_c (generic function with 1 method)

Then, I define a function to get $(C_s, S_s)$ that maximizes
\begin{equation*}
\frac{C_s^{1 - \rho}}{1 - \rho} + \beta \sum_{k = 1}^K w_k \left[ \sum_{j = 1}^n c_j \phi_j((W_s - C_s - S_s)^{\beta_m} \varepsilon_k' + Z_k' + (1 + r) S_s) \right].
\end{equation*}


In [36]:
function jump_func(W::AbstractFloat,
        c::AbstractArray{Float64,1},
        basis::Dict,
        epsilon::AbstractVector{Float64},
        Z::AbstractVector{Float64},
        w::AbstractVector{Float64},
        rho::AbstractFloat,
        beta::AbstractFloat,
        beta_m::AbstractFloat,
        r::AbstractFloat)

    m = Model(with_optimizer(Ipopt.Optimizer, print_level=0))
    @variable(m, C >= 0.01)
    @variable(m, S >= 0.01)
    function calculate_vs_inner(C, S)
        return calculate_vs(C, S, W, c, basis, epsilon, Z, w, rho, beta, beta_m, r)
    end
    JuMP.register(m, :calculate_vs_inner, 2, calculate_vs_inner, autodiff = true)
    @NLobjective(m, Max, calculate_vs_inner(C, S))
    @NLconstraint(m, W - C - S >= 0.01)
    JuMP.optimize!(m)
    return JuMP.value(C), JuMP.value(S), objective_value(m)
end

jump_func (generic function with 1 method)

In [37]:
# Define a function to calculate the value function given (consumption, saving).
function get_optimal_C_S(W, c, basis, epsilon, Z, w, rho, beta, beta_m, r, di_v)
    # set output vectors
    vs_new = zeros(di_v);
    C_new = zeros(di_v);
    S_new = zeros(di_v);       

    # for each collocation node, solve the optimization problem
    for s in 1:di_v
        C_new[s], S_new[s], vs_new[s] = jump_func(W[s], c, basis, epsilon, Z, w, rho, beta, beta_m, r);
    end
    return C_new, S_new, vs_new 
end

get_optimal_C_S (generic function with 1 method)

Finally, I define a function to obtain policy functions given parameters.
Here, I set the following values for the collocation method and the Gaussian quadrature approximation:

- I approximate the value function with cubic splines with 50 nodes (`di_v`), with an interval $(0.1, 20.0)$ (`W_left_end` and `W_right_end`).
For approximations of policy functions, I use the same nodes and interval.
- I approximate the expectation using the Gaussian quadrature approximation, with 8 nodes for $\varepsilon$ (`di_epsilon`) and 8 nodes for $Z$ (`di_Z`).

In [50]:
function policy_function_iteration(rho,
                                beta,
                                beta_m::Float64,
                                mu_epsilon::Float64,
                                sigma_epsilon::Float64,
                                mu_Z::Float64,
                                sigma_Z::Float64,
                                r::Float64,
                                W_left_end::Float64=0.1,
                                W_right_end::Float64=20.0,
                                di_v::Integer=50,
                                di_epsilon::Integer=8,
                                di_Z::Integer=8
                                )
    
    # Collocation coefficients and basis functions for value functions
    basis = fundefn(:spli, di_v, W_left_end, W_right_end);
    # Initial guess of value function = log(W)
    W = funnode(basis)[1];
    y = log.(W);
    c = funfitxy(basis, W, y)[1];
    # Initial guesses of C and S = 1/3 W
    C = 1/3 * W;
    S = 1/3 * W;
    # Gaussian quadrature nodes and weights for epsilon and Z
    a, w = qnwlogn([di_epsilon, di_Z], [mu_epsilon, mu_Z], [sigma_epsilon^2 0.0; 0.0 sigma_Z^2]);
    epsilon, Z = a[:,1], a[:,2];

    # Calculate the value function given initial guess of (consumption, saving).
    start = time()
    future_v = zeros(length(C))
    for k in 1:length(epsilon)
        future_v = future_v .+ beta * funeval(c, basis, [max.(W - C - S, 0.01) .^ beta_m * epsilon[k] .+ Z[k] + (1 + r) * S])[1] .* w[k]
    end
    vs = C .^ (1 - rho) / (1 - rho) .+ future_v[:,1]
    elapsed = time() - start
#     print("Time elapsed for value function calculation: $elapsed \n")
    
    # Policy function iteration
    start_policy = time();
    error = 1e10;
    i = 1;
    tol = 1e-5;
    max_iter = 100;

    while (error > tol) && (i < max_iter)
        # update collocation coefficients
        c = update_c(W, vs, basis);
        # obtain optimal actions given the updated collocation coefficients
        start_time = time();
        C_new, S_new, vs_new = get_optimal_C_S(W, c, basis, epsilon, Z, w, rho, beta, beta_m, r, di_v);
        elapsed = time() - start_time;
#         print("Time elapsed for iteration $i: $elapsed \n")
        error = max(maximum(abs, C_new - C), maximum(abs, S_new - S))
        i += 1;
        C = C_new;
        S = S_new;
        vs = vs_new;
        #print(DataFrame(C = C, S = S, M = W - C - S, vs = vs, W = W))
        #print("iteration = $i, error = $error \n")
    end
    elapsed_policy = time() - start_policy
#     print("Time elapsed for policy function iteration: $elapsed_policy \n")
    return C, S, vs, W
end


policy_function_itartion (generic function with 12 methods)

## Data simulation

Here, I generate a dataset to be used for estimation.
For this, I randomly generate the wealth of the first period, and obtain

- consumption in the first period;
- saving in the first period;
- business revenue in the second period;
- gross interest income in the second period;
- wealth in the second period.

In reality, the dataset can contain more information (eg. business revenue and interest income in the first period) and more times periods, but the additional information is not needed for estimation.
The (true) parameters are set as followings:

- $\rho = 1.75$;
- $\beta = 0.9$;
- $\beta_m = 0.8$;
- $\mu_{\varepsilon} = 0.50$;
- $\sigma_{\varepsilon} = 0.05$;
- $\mu_Z = 0.02$;
- $\sigma_Z = 0.01$;
- $r = 0.01$.

The number of households is set to be 1000.

In [52]:
# Generate a two-period dataset =================

# The number of household
nHH = 1000;

# Parameter values
rho = 1.75;
beta = 0.9;
beta_m = 0.8;
mu_epsilon = 0.50;
sigma_epsilon = 0.05;
mu_Z = 0.02;
sigma_Z = 0.01;
r = 0.01;

# Optimal actions at each colloation node
@time C_opt, S_opt, vs_opt, W_grid = policy_function_iteration(rho, beta, beta_m, mu_epsilon, sigma_epsilon, mu_Z, sigma_Z, r);

# Obtain collocation coefficients for policy functions
W_left_end = 0.1;
W_right_end = 20.0;
di_v = 50;
basis = fundefn(:spli, di_v, W_left_end, W_right_end);
W = funnode(basis)[1];
c = funfitxy(basis, W, vs_opt)[1];
c_C = funfitxy(basis, W, C_opt)[1];
c_S = funfitxy(basis, W, S_opt)[1];


 18.737549 seconds (5.48 M allocations: 2.077 GiB, 2.60% gc time)


In [53]:
# Randomly generate wealth in the first period (uniformly distributed between 0.1 and 18)
Random.seed!(1234);
W_first = rand(nHH) .* (18.0 - 0.1) .+ 0.1;

# Randomly generate productivity shock (epsilon) and income shock (Z)
epsilon_second = exp.(mu_epsilon .+ sigma_epsilon .* randn(nHH));
Z_second = exp.(mu_Z .+ sigma_Z .* randn(nHH));

# Obtain first-period consumption, saving, and business input
C_first = funeval(c_C, basis, W_first)[1][:,1];
S_first = funeval(c_S, basis, W_first)[1][:,1];
M_first = max.(W_first - C_first - S_first, 0.01);

# Calculate second-period business revenue
Y_second = (M_first .^ beta_m) .* epsilon_second;

# Calculate second-period interest income
int_inc_second = (1 + r) .* (W_first .- C_first .- M_first);

# Calculate second-period wealth
W_second = Y_second .+ Z_second .+ int_inc_second;

# Construct a dataset

sim_data = DataFrame(W_first = W_first,
                    C_first = C_first,
                    M_first = M_first,
                    Y_second = Y_second,
                    int_inc_second = int_inc_second,
                    W_second = W_second);

# I add a small noise to variables in the dataset (othewise, r would be exactly estimated.)
#sim_data = sim_data .* (rand(nrow(sim_data), ncol(sim_data)) * ((1.0 + 1e-7) - (1 - 1e-7)) .+ (1 - 1e-7))


## Estimation

Here, I define functions to estimate parameters.
First, I define a function to estimate $\theta_2$.

In [54]:
# Define a function to estimate theta_2

function estimate_theta2(data)
    
    # estimation of beta_m, mu_epsilon, and sigma_epsilon
    reg_Y_M_coef = llsq(hcat(convert(Array, log.(data[:,:M_first]))),log.(data[:,:Y_second]));
    beta_m_hat, mu_epsilon_hat = reg_Y_M_coef;
    sigma_epsilon_hat = std(log.(data[:,:Y_second]) .- log.(data[:,:M_first]) * beta_m_hat .- mu_epsilon_hat);
    
    # estimation of r
    r_hat = mean(data[:,:int_inc_second] ./ (data[:,:W_first] - data[:,:M_first] - data[:,:C_first])) - 1.0;

    # estimation of mu_Z and sigma_Z
    Z_first_est = data[:,:W_second] - data[:,:Y_second] - data[:,:int_inc_second];
    mu_Z_hat = mean(log.(Z_first_est));
    sigma_Z_hat = std(log.(Z_first_est));
    
    return [beta_m_hat, mu_epsilon_hat, sigma_epsilon_hat, mu_Z_hat, sigma_Z_hat, r_hat]
end

estimate_theta2 (generic function with 1 method)

Next, I define a function to calculate the distance $D(\theta_1, \theta_2)$.

In [55]:
# Define a function to estimate the distance between model-predicted moments and data moments

function calculate_distance(theta_1, theta_2, data,
                            W_left_end::Float64=0.1,
                            W_right_end::Float64=20.0,
                            di_v::Integer=50)
    
    print([theta_1; theta_2], '\n')
    # expand theta_1 and theta_2
    
    rho = theta_1[1];
    beta = theta_1[2];
    
    beta_m = theta_2[1];
    mu_epsilon = theta_2[2];
    sigma_epsilon = theta_2[3];
    mu_Z = theta_2[4];
    sigma_Z = theta_2[5];
    r = theta_2[6];
   
    # Optimal actions at each colloation node
    C_opt, S_opt, vs_opt, W_grid = policy_function_iteration(rho, beta, beta_m, mu_epsilon, sigma_epsilon, mu_Z, sigma_Z, r);

    # Obtain collocation coefficients for policy functions
    basis = fundefn(:spli, di_v, W_left_end, W_right_end);
    W = funnode(basis)[1];
    c_C = funfitxy(basis, W, C_opt)[1];
    c_S = funfitxy(basis, W, S_opt)[1];

    C_first_model = funeval(c_C, basis, data[:,:W_first])[1];
    S_first_model = funeval(c_S, basis, data[:,:W_first])[1]; 
    
    # construct distance
    
    distance = dot(C_first_model - sim_data[:,:C_first], C_first_model - sim_data[:,:C_first]) + 
                dot(S_first_model - (sim_data[:,:W_first] .- sim_data[:,:C_first] .- sim_data[:,:M_first]), S_first_model - (sim_data[:,:W_first] .- sim_data[:,:C_first] .- sim_data[:,:M_first]));
    print(distance, '\n')
    return distance
end

calculate_distance (generic function with 4 methods)

Next, I define a function to estimate $\theta_1$ given $\theta_2$.

In [59]:
# Define a function to estimate theta_1 given theta_2

function estimate_theta1(data, theta_2, theta_1_init)
    # find theta_1 that minimizes distance
    optim_output = optimize(x -> calculate_distance([x[1], x[2]], theta_2, data), theta_1_init);
    return optim_output.minimizer   
end
    

estimate_theta1 (generic function with 2 methods)

In [56]:
# # Define a function to estimate theta_1 given theta_2

# function estimate_theta1(data::DataFrame, 
#                         theta_2::AbstractVector{Float64})
    
#     m = Model(with_optimizer(Ipopt.Optimizer, print_level=0))
#     @variable(m, rho >= 1.01)
#     @variable(m, beta <= 0.99)
#     function calculate_distance_inner(rho, beta)
#         return calculate_distance([rho, beta], theta_2, data)
#     end
#     JuMP.register(m, :calculate_distance_inner, 2, calculate_distance_inner, autodiff = true)
#     @NLobjective(m, Min, calculate_distance_inner(rho, beta))
#     JuMP.optimize!(m)
#     print(objective_value(m), '\n')
#     return [JuMP.value(rho), JuMP.value(beta)]
# end


estimate_theta1 (generic function with 1 method)

Finally, I define a function to estimate all parameters given data and an initial guess for $\theta_1$.

In [60]:
# Define a function to estimate all parameters given data and an initial guessfor theta_1

function estimation_all(data, theta_1_init)
    # estimate theta_2
    theta_2_hat = estimate_theta2(data);
    
    # estimate theta_1
    theta_1_hat = estimate_theta1(data, theta_2_hat, theta_1_init)
#     theta_1_hat = estimate_theta1(data, theta_2_hat)
    
    return [theta_1_hat; collect(theta_2_hat)]
end
    

estimation_all (generic function with 1 method)

In [61]:
@time theta_hat = estimation_all(sim_data, [1.75, 0.9] * 0.9)
writedlm( "temp/theta_hat.csv",  theta_hat, ',')

[1.575, 0.81, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
2188.088856945582
[1.575, 0.81, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
2188.088856945582
[2.3875, 0.81, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
2919.3989322891925
[1.575, 1.24, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
14396.511956419425
[2.3875, 0.38, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
11684.714911509382
[2.18437, 0.595, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
7672.2681495557
[1.77813, 1.025, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
15442.136583815056
[2.08281, 0.7025, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
5202.032074583339
[1.87969, 0.9175, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
311.9612205679106
[1.77813, 1.025, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
17484.30196292618
[1.06719, 0.9175, 0.799491, 0.501809, 0.050417, 0.0198648, 0.01021, 0.01]
226.70725569010403
[0.407

In [68]:
[rho, beta, beta_m, mu_epsilon, sigma_epsilon, mu_Z, sigma_Z, r]

8-element Array{Float64,1}:
 1.75
 0.9 
 0.8 
 0.5 
 0.05
 0.02
 0.01
 0.01

In [70]:
parameter_table = DataFrame(Name = ["rho", "beta", "beta_m", "mu_epsilon", "sigma_epsilon", "mu_Z", "sigma_Z", "r"],
                            True = [rho, beta, beta_m, mu_epsilon, sigma_epsilon, mu_Z, sigma_Z, r],
                            Estimates = theta_hat[:,1])

,Name,True,Estimates
,String,Float64,Float64
1,rho,1.75,1.75038
2,beta,0.9,0.90077
3,beta_m,0.8,0.799491
4,mu_epsilon,0.5,0.501809
5,sigma_epsilon,0.05,0.050417
6,mu_Z,0.02,0.0198648
7,sigma_Z,0.01,0.01021
8,r,0.01,0.01


## Counterfactual simulation

Here, I conduct a counterfactual simulation.
Specifically, I analyze the effect of cash transfer on consumption, business inputs, interest income, business revenue, and the second-period wealth.
Cash transfer increases wealth in the first period, which affects various economic activities of business owners.
In this experiment, I increase wealth by 1.

For simulation, I need to generate random variables (productivity shocks and income shocks).
I generate these random variables 10 times and take averages of the resulting variables as values under a counterfactual simulation.

In [71]:
theta_hat = readdlm("temp/theta_hat.csv");

In [72]:
rho = theta_hat[1];
beta = theta_hat[2];    
beta_m = theta_hat[3];
mu_epsilon = theta_hat[4];
sigma_epsilon = theta_hat[5];
mu_Z = theta_hat[6];
sigma_Z = theta_hat[7];
r = theta_hat[8];

# Optimal actions at each colloation node
C_opt, S_opt, vs_opt, W_grid = policy_function_itartion(rho, beta, beta_m, mu_epsilon, sigma_epsilon, mu_Z, sigma_Z, r);

# Obtain collocation coefficients for policy functions
W_left_end = 0.1;
W_right_end = 20.0;
di_v = 50;
basis = fundefn(:spli, di_v, W_left_end, W_right_end);
W = funnode(basis)[1];
c = funfitxy(basis, W, vs_opt)[1];
c_C = funfitxy(basis, W, C_opt)[1];
c_S = funfitxy(basis, W, S_opt)[1];

In [73]:
minimum(W_first)


0.11289268035460083

In [78]:
W_first_cf = W_first .+ 1;

num_sim = 10;

# Randomly generate productivity shock (epsilon) and income shock (Z) 10 times
epsilon_second = exp.(mu_epsilon .+ sigma_epsilon .* randn(nHH, num_sim));
Z_second = exp.(mu_Z .+ sigma_Z .* randn(nHH, num_sim));

# Obtain first-period consumption, saving, and business input
C_first_cf = funeval(c_C, basis, W_first_cf)[1][:,1];
S_first_cf = funeval(c_S, basis, W_first_cf)[1][:,1];
M_first_cf = max.(W_first_cf - C_first_cf - S_first_cf, 0.01);

# Calculate second-period business revenue
Y_second_cf = (M_first_cf .^ beta_m) .* epsilon_second;

# Calculate second-period interest income
int_inc_second_cf = (1 + r) .* (W_first_cf .- C_first_cf .- M_first_cf);

# Calculate second-period wealth
W_second_cf = Y_second_cf .+ Z_second .+ int_inc_second_cf;


In [79]:
mean(C_first_cf), mean(M_first_cf), mean(Y_second_cf), mean(int_inc_second_cf), mean(W_second_cf)

(2.967558925649922, 3.0738618509983042, 4.014317256436098, 3.9916950913566907, 9.02623129722367)

In [80]:
mean(C_first), mean(M_first), mean(Y_second), mean(int_inc_second), mean(W_second)

(2.76364148944142, 2.856510112451665, 3.741619074124894, 3.407176957859385, 8.168912557496355)

In [81]:
df = DataFrame(Name = ["C_first", "M_first", "Y_second", "int_inc_second", "W_second"],
            Data = [mean(C_first), mean(M_first), mean(Y_second), mean(int_inc_second), mean(W_second)],
            Simulation = [mean(C_first_cf), mean(M_first_cf), mean(Y_second_cf), mean(int_inc_second_cf), mean(W_second_cf)])
df.ATE = df[:,:Simulation] - df[:,:Data];
df

,Name,Data,Simulation,ATE
,String,Float64,Float64,Float64
1,C_first,2.76364,2.96756,0.203917
2,M_first,2.85651,3.07386,0.217352
3,Y_second,3.74162,4.01432,0.272698
4,int_inc_second,3.40718,3.9917,0.584518
5,W_second,8.16891,9.02623,0.857319
